# Library import

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import RandomOverSampler

import sys
sys.path.append("C:/Users/Walraff/OneDrive - Universite de Liege/Documents/Ulg/Master2/TFE/")
import utils

In [10]:
data_path = "C:/Users/Walraff/OneDrive - Universite de Liege/Documents/Ulg/Master2/TFE/data"
# data_path = "/content/drive/MyDrive/TFE"
original_df = pd.read_csv(f'{data_path}/final_status_SPARE.csv')
original_df

,ProteinName_SPARE,Peptide_SPARE,Status_SPARE
0,sp|P02751|FINC_HUMAN,VDVIPVNLPGEHGQR,bon
1,sp|P02751|FINC_HUMAN,STTPDITGYR,bon
2,sp|P02751|FINC_HUMAN,SYTITGLQPGTDYK,bon
3,sp|P02751|FINC_HUMAN,IYLYTLNDNAR,bon
4,sp|P04114|APOB_HUMAN,TGISPLALIK,bon
...,...,...,...
150,sp|P02743|SAMP_HUMAN,VGEYSLYIGR,bon
151,sp|P04004|VTNC_HUMAN,GQYCYELDEK,mauvais
152,sp|P04004|VTNC_HUMAN,FEDGVLDPDYPR,bon
153,sp|P04004|VTNC_HUMAN,DWHGVPGQVDAAMAGR,bon


# Functions and loading data

In [ ]:
# Creating a dataframe with the sequences and labels
df = pd.DataFrame()
df["sequence"] = original_df["Peptide_SPARE"]
df["quantotypic"] = original_df.apply(lambda row: 0 if row['Status_SPARE'] == 'bon' else 1, axis=1)

positive_df = df[df['quantotypic'] == 0]
negative_df = df[df['quantotypic'] == 1]

class_counts = df['quantotypic'].value_counts()
max_len = df['sequence'].str.len().max()
pos_weight = class_counts[0] / class_counts[1]
print(class_counts)
print(pos_weight)

quantotypic
0    117
1     38
Name: count, dtype: int64
3.0789473684210527


In [ ]:
def one_hot_encode_sequence(sequence, max_len, vocab):
    """
    Converts an amino acid sequence into a one-hot encoded matrix of shape (max_len, vocab_size).
    Unused positions (due to padding) remain as zero rows.

    Args:
        sequence (str): Amino acid sequence.
        max_len (int): Maximum sequence length (for padding/truncation).
        vocab (dict): Mapping from amino acid to index (1-based).

    Returns:
        np.ndarray: One-hot encoded matrix of shape (max_len, len(vocab)).
    """
    n_vocab = len(vocab)
    encoded = np.zeros((max_len, n_vocab), dtype=np.float32)

    for i, aa in enumerate(sequence):
        if i < max_len:
            idx = vocab.get(aa, None)
            if idx is not None:
                encoded[i, idx - 1] = 1.0  # Convert to 0-based index

    return encoded

amino_acid_vocab = {aa: idx+1 for idx, aa in enumerate("ACDEFGHIKLMNPQRSTVWY")}

df['encoded_sequence'] = df['sequence'].apply(lambda seq: one_hot_encode_sequence(seq, max_len, amino_acid_vocab))

# Random Forest

In [ ]:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
count = 1

# Trying with and without weighting
for weighting in [False, True]:
    temp_result_list = []
    # loop through the folds
    for fold, (train_idx, test_idx) in enumerate(kf.split(df['encoded_sequence'], df['quantotypic'])):
        # Splitting the data into train and test sets
        train_df = df.iloc[train_idx]
        test_df = df.iloc[test_idx]
        print(f"Fold: {fold}")
        print()
        
        # Prepare the data for training and testing
        X_train = np.stack(train_df['encoded_sequence'].values)
        X_train = X_train.reshape(X_train.shape[0], -1)
        y_train = train_df['quantotypic'].values

        X_test = np.stack(test_df['encoded_sequence'].values)
        X_test = X_test.reshape(X_test.shape[0], -1)
        y_test = test_df['quantotypic'].values

        # Param grid for RandomForestClassifier
        param_grid = {
            'max_depth': [None, 1, 3, 5, 6],
            'max_features': ['sqrt', 'log2', None], # None => max feature = n features
        }

        # if weighting is True, add class_weight to the classifier
        if weighting:
            rf = RandomForestClassifier(n_estimators=1000, random_state=42, class_weight='balanced')
        else:
            rf = RandomForestClassifier(n_estimators=1000, random_state=42)

        # Train the model using GridSearchCV
        grid_search_rf = GridSearchCV(rf, param_grid, cv=5, scoring='average_precision', verbose=4, n_jobs=-1)
        grid_search_rf.fit(X_train, y_train)

        print(grid_search_rf.best_params_)
        print(grid_search_rf.best_score_)

        # Retrieve the best parameters
        best_params = grid_search_rf.best_params_

        # Initialize a new RandomForestClassifier with the best parameters
        if weighting:
            best_rf = RandomForestClassifier(n_estimators=1000,
                **best_params,
                random_state=42,
                class_weight='balanced' 
            )
        else:
            best_rf = RandomForestClassifier(n_estimators=1000,
                **best_params,
                random_state=42 
            )

        # Retrain the model on the entire training set
        best_rf.fit(X_train, y_train)

        # Predict on the test set
        y_pred = best_rf.predict(X_test)

        # Evaluate the model using the provided eval function
        accuracy, precision, recall, f1, roc_auc, pr_auc = utils.eval(y_test, y_pred)

        metrics = {
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1": f1,
            "roc_auc": roc_auc,
            "pr_auc": pr_auc
        }

        temp_result_list.append(metrics)

        # Print the results
        print(f"Accuracy: {accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1 Score: {f1:.4f}")
        print(f"ROC AUC: {roc_auc:.4f}")
        print(f"PR AUC: {pr_auc:.4f}")

    accuracy_list = [result['accuracy'] for result in temp_result_list]
    precision_list = [result['precision'] for result in temp_result_list]
    recall_list = [result['recall'] for result in temp_result_list]
    f1_list = [result['f1'] for result in temp_result_list]
    roc_auc_list = [result['roc_auc'] for result in temp_result_list]
    pr_auc_list = [result['pr_auc'] for result in temp_result_list]

    metrics_summary = {
        "accuracy_mean": np.mean(accuracy_list),
        "accuracy_std": np.std(accuracy_list),
        "precision_mean": np.mean(precision_list),
        "precision_std": np.std(precision_list),
        "recall_mean": np.mean(recall_list),
        "recall_std": np.std(recall_list),
        "f1_mean": np.mean(f1_list),
        "f1_std": np.std(f1_list),
        "roc_auc_mean": np.mean(roc_auc_list),
        "roc_auc_std": np.std(roc_auc_list),
        "pr_auc_mean": np.mean(pr_auc_list),
        "pr_auc_std": np.std(pr_auc_list)
    }

    dict_save = {
        "weighting": weighting,
        "oversampling": False,
        "metrics_summary": metrics_summary
    }

    path = "C:/Users/Walraff/OneDrive - Universite de Liege/Documents/Ulg/Master2/TFE/Results/ValidProtocol"
    utils.write_into_json(dict_save, f"{path}/rf_scratch_experiment_{count}.json")
    count+=1

Fold: 0

Fitting 5 folds for each of 15 candidates, totalling 75 fits
{'max_depth': 3, 'max_features': None}
0.3836180150339142
Accuracy: 0.8065
Precision: 1.0000
Recall: 0.1429
F1 Score: 0.2500
ROC AUC: 0.5714
PR AUC: 0.3364
Fold: 1

Fitting 5 folds for each of 15 candidates, totalling 75 fits
{'max_depth': 1, 'max_features': 'log2'}
0.3507694363231736


c:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.7742
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000
ROC AUC: 0.5000
PR AUC: 0.2258
Fold: 2

Fitting 5 folds for each of 15 candidates, totalling 75 fits
{'max_depth': 5, 'max_features': 'sqrt'}
0.4503078580508248


c:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.7419
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000
ROC AUC: 0.5000
PR AUC: 0.2581
Fold: 3

Fitting 5 folds for each of 15 candidates, totalling 75 fits
{'max_depth': 1, 'max_features': None}
0.4001824269641608
Accuracy: 0.7742
Precision: 0.6667
Recall: 0.2500
F1 Score: 0.3636
ROC AUC: 0.6033
PR AUC: 0.3602
Fold: 4

Fitting 5 folds for each of 15 candidates, totalling 75 fits
{'max_depth': None, 'max_features': None}
0.47287215812983063
Accuracy: 0.6774
Precision: 0.2500
Recall: 0.1250
F1 Score: 0.1667
ROC AUC: 0.4973
PR AUC: 0.2571
Fold: 0

Fitting 5 folds for each of 15 candidates, totalling 75 fits
{'max_depth': 5, 'max_features': 'log2'}
0.3578912737012895
Accuracy: 0.7419
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000
ROC AUC: 0.4792
PR AUC: 0.2258
Fold: 1

Fitting 5 folds for each of 15 candidates, totalling 75 fits
{'max_depth': 3, 'max_features': 'log2'}
0.3439316717606624
Accuracy: 0.7742
Precision: 0.5000
Recall: 0.1429
F1 Score: 0.2222
ROC AUC: 0.5506
PR AU

c:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# New loop with oversampling using ImbPipeline
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

temp_result_list = []
# loop over folds
for fold, (train_idx, test_idx) in enumerate(kf.split(df['encoded_sequence'], df['quantotypic'])):
    # Splitting the data into train and test sets
    train_df = df.iloc[train_idx]
    test_df = df.iloc[test_idx]
    print(f"Fold: {fold}")
    print()

    # Prepare the data for training and testing
    X_train = np.stack(train_df['encoded_sequence'].values)
    X_train = X_train.reshape(X_train.shape[0], -1)
    y_train = train_df['quantotypic'].values

    X_test = np.stack(test_df['encoded_sequence'].values)
    X_test = X_test.reshape(X_test.shape[0], -1)
    y_test = test_df['quantotypic'].values

    # Param grid for RandomForestClassifier
    param_grid = {
    'classifier__max_depth': [None, 1, 3, 5, 6],
    'classifier__max_features': ['sqrt', 'log2', None],  # None = utiliser toutes les features
    }

    # Create a pipeline with RandomOverSampler and RandomForestClassifier
    pipeline = ImbPipeline([
        ('oversampler', RandomOverSampler(random_state=42)), 
        ('classifier', RandomForestClassifier(
            n_estimators=1000, 
            random_state=42
        ))
    ])

    # Training the model using GridSearchCV
    grid_search_rf = GridSearchCV(
        estimator=pipeline,
        param_grid=param_grid,
        cv=5,  
        scoring='average_precision',
        verbose=4, 
        n_jobs=-1 
    )

    grid_search_rf.fit(X_train, y_train)

    print(grid_search_rf.best_params_)
    print(grid_search_rf.best_score_)

    # Retrieve the best parameters
    best_params = grid_search_rf.best_params_

    best_rf = RandomForestClassifier(
        n_estimators=1000, 
        **{k.split('__')[1]: v for k, v in best_params.items()},  
        random_state=42
    )

    # Retrain the model on the entire training set with oversampling
    oversampler = RandomOverSampler(random_state=42)
    X_resampled, y_resampled = oversampler.fit_resample(X_train, y_train)
    best_rf.fit(X_resampled, y_resampled)

    # Predict on the test set
    y_pred = best_rf.predict(X_test)

    # Evaluate the model using the provided eval function
    accuracy, precision, recall, f1, roc_auc, pr_auc = utils.eval(y_test, y_pred)

    metrics = {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "roc_auc": roc_auc,
        "pr_auc": pr_auc
    }

    temp_result_list.append(metrics)

    # Print the results
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"ROC AUC: {roc_auc:.4f}")
    print(f"PR AUC: {pr_auc:.4f}")

accuracy_list = [result['accuracy'] for result in temp_result_list]
precision_list = [result['precision'] for result in temp_result_list]
recall_list = [result['recall'] for result in temp_result_list]
f1_list = [result['f1'] for result in temp_result_list]
roc_auc_list = [result['roc_auc'] for result in temp_result_list]
pr_auc_list = [result['pr_auc'] for result in temp_result_list]

metrics_summary = {
    "accuracy_mean": np.mean(accuracy_list),
    "accuracy_std": np.std(accuracy_list),
    "precision_mean": np.mean(precision_list),
    "precision_std": np.std(precision_list),
    "recall_mean": np.mean(recall_list),
    "recall_std": np.std(recall_list),
    "f1_mean": np.mean(f1_list),
    "f1_std": np.std(f1_list),
    "roc_auc_mean": np.mean(roc_auc_list),
    "roc_auc_std": np.std(roc_auc_list),
    "pr_auc_mean": np.mean(pr_auc_list),
    "pr_auc_std": np.std(pr_auc_list)
}

dict_save = {
    "weighting": False,
    "oversampling": True,
    "metrics_summary": metrics_summary
}

path = "C:/Users/Walraff/OneDrive - Universite de Liege/Documents/Ulg/Master2/TFE/Results/ValidProtocol"
utils.write_into_json(dict_save, f"{path}/rf_scratch_experiment_3.json")

Fold: 0

Fitting 5 folds for each of 15 candidates, totalling 75 fits
{'classifier__max_depth': 1, 'classifier__max_features': None}
0.3651302866753802
Accuracy: 0.8387
Precision: 0.7500
Recall: 0.4286
F1 Score: 0.5455
ROC AUC: 0.6935
PR AUC: 0.4505
Fold: 1

Fitting 5 folds for each of 15 candidates, totalling 75 fits
{'classifier__max_depth': 1, 'classifier__max_features': 'sqrt'}
0.3830598013400399
Accuracy: 0.6774
Precision: 0.2000
Recall: 0.1429
F1 Score: 0.1667
ROC AUC: 0.4881
PR AUC: 0.2221
Fold: 2

Fitting 5 folds for each of 15 candidates, totalling 75 fits
{'classifier__max_depth': 1, 'classifier__max_features': None}
0.4569403037050096
Accuracy: 0.4194
Precision: 0.2500
Recall: 0.6250
F1 Score: 0.3571
ROC AUC: 0.4864
PR AUC: 0.2530
Fold: 3

Fitting 5 folds for each of 15 candidates, totalling 75 fits
{'classifier__max_depth': 1, 'classifier__max_features': None}
0.4055869185280949
Accuracy: 0.7097
Precision: 0.4444
Recall: 0.5000
F1 Score: 0.4706
ROC AUC: 0.6413
PR AUC: 0.351

# XGBoost

In [ ]:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
count = 1

# Trying with and without weighting
for weighting in [False, True]:
    temp_result_list = []

    # loop over the folds
    for fold, (train_idx, test_idx) in enumerate(kf.split(df['encoded_sequence'], df['quantotypic'])):
        # Splitting the data into train and test sets
        train_df = df.iloc[train_idx]
        test_df = df.iloc[test_idx]
        print(f"Fold: {fold}\n")

        # Prepare the data for training and testing
        X_train = np.stack(train_df['encoded_sequence'].values)
        X_train = X_train.reshape(X_train.shape[0], -1)
        y_train = train_df['quantotypic'].values

        X_test = np.stack(test_df['encoded_sequence'].values)
        X_test = X_test.reshape(X_test.shape[0], -1)
        y_test = test_df['quantotypic'].values

        # Param grid for XGBClassifier
        param_grid = {
            'max_depth': [None, 1, 3, 5, 6],             
            'learning_rate': [0.01, 0.1, 0.2, 0.3, 0.4], 
        }

        # if weighting is True, add scale_pos_weight to the classifier
        if weighting:
            model = XGBClassifier(
                n_estimators=1000,
                scale_pos_weight=pos_weight, 
                random_state=42
            )
        else:
            model = XGBClassifier(
                n_estimators=1000,
                random_state=42
            )

        # Train the model using GridSearchCV
        grid_search = GridSearchCV(
            model, param_grid, cv=5, scoring="average_precision", verbose=4, n_jobs=-1
        )
        grid_search.fit(X_train, y_train)

        print(grid_search.best_params_)
        print(grid_search.best_score_)

        # Retrieve the best parameters
        best_params = grid_search.best_params_

        # Initialize a new XGBClassifier with the best parameters
        if weighting:
            best_model = XGBClassifier(
                n_estimators=1000,
                scale_pos_weight=pos_weight,
                random_state=42,
                **best_params
            )
        else:
            best_model = XGBClassifier(
                n_estimators=1000,
                random_state=42,
                **best_params
            )

        # Retrain the model on the entire training set
        best_model.fit(X_train, y_train)

        # Predict on the test set
        y_pred = best_model.predict(X_test)

        # Evaluate the model using the provided eval function
        accuracy, precision, recall, f1, roc_auc, pr_auc = utils.eval(y_test, y_pred)

        metrics = {
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1": f1,
            "roc_auc": roc_auc,
            "pr_auc": pr_auc
        }
        
        temp_result_list.append(metrics)

        # Print the results
        print(f"Accuracy: {accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1 Score: {f1:.4f}")
        print(f"ROC AUC: {roc_auc:.4f}")
        print(f"PR AUC: {pr_auc:.4f}")

    accuracy_list = [result['accuracy'] for result in temp_result_list]
    precision_list = [result['precision'] for result in temp_result_list]
    recall_list = [result['recall'] for result in temp_result_list]
    f1_list = [result['f1'] for result in temp_result_list]
    roc_auc_list = [result['roc_auc'] for result in temp_result_list]
    pr_auc_list = [result['pr_auc'] for result in temp_result_list]

    metrics_summary = {
        "accuracy_mean": np.mean(accuracy_list),
        "accuracy_std": np.std(accuracy_list),
        "precision_mean": np.mean(precision_list),
        "precision_std": np.std(precision_list),
        "recall_mean": np.mean(recall_list),
        "recall_std": np.std(recall_list),
        "f1_mean": np.mean(f1_list),
        "f1_std": np.std(f1_list),
        "roc_auc_mean": np.mean(roc_auc_list),
        "roc_auc_std": np.std(roc_auc_list),
        "pr_auc_mean": np.mean(pr_auc_list),
        "pr_auc_std": np.std(pr_auc_list)
    }

    # Sauvegarde des résultats
    dict_save = {
        "weighting": weighting,
        "oversampling": False,
        "metrics_summary": metrics_summary
    }

    path = "C:/Users/Walraff/OneDrive - Universite de Liege/Documents/Ulg/Master2/TFE/Results/ValidProtocol"
    utils.write_into_json(dict_save, f"{path}/xgb_scratch_experiment_{count}.json")
    count += 1

Fold: 0

Fitting 5 folds for each of 25 candidates, totalling 125 fits
{'learning_rate': 0.2, 'max_depth': 1}
0.4104033594580005
Accuracy: 0.6774
Precision: 0.3333
Recall: 0.4286
F1 Score: 0.3750
ROC AUC: 0.5893
PR AUC: 0.2719
Fold: 1

Fitting 5 folds for each of 25 candidates, totalling 125 fits
{'learning_rate': 0.01, 'max_depth': 3}
0.36592512307713354
Accuracy: 0.7419
Precision: 0.3333
Recall: 0.1429
F1 Score: 0.2000
ROC AUC: 0.5298
PR AUC: 0.2412
Fold: 2

Fitting 5 folds for each of 25 candidates, totalling 125 fits
{'learning_rate': 0.3, 'max_depth': 3}
0.39804043737480266
Accuracy: 0.7742
Precision: 0.6000
Recall: 0.3750
F1 Score: 0.4615
ROC AUC: 0.6440
PR AUC: 0.3863
Fold: 3

Fitting 5 folds for each of 25 candidates, totalling 125 fits
{'learning_rate': 0.4, 'max_depth': 1}
0.386684581971027
Accuracy: 0.6129
Precision: 0.1667
Recall: 0.1250
F1 Score: 0.1429
ROC AUC: 0.4538
PR AUC: 0.2466
Fold: 4

Fitting 5 folds for each of 25 candidates, totalling 125 fits
{'learning_rate': 0

c:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.7419
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000
ROC AUC: 0.5000
PR AUC: 0.2581
Fold: 0

Fitting 5 folds for each of 25 candidates, totalling 125 fits
{'learning_rate': 0.3, 'max_depth': 1}
0.42218235743709764
Accuracy: 0.6452
Precision: 0.3333
Recall: 0.5714
F1 Score: 0.4211
ROC AUC: 0.6190
PR AUC: 0.2873
Fold: 1

Fitting 5 folds for each of 25 candidates, totalling 125 fits
{'learning_rate': 0.3, 'max_depth': 5}
0.3887694296912996
Accuracy: 0.7742
Precision: 0.5000
Recall: 0.4286
F1 Score: 0.4615
ROC AUC: 0.6518
PR AUC: 0.3433
Fold: 2

Fitting 5 folds for each of 25 candidates, totalling 125 fits
{'learning_rate': 0.01, 'max_depth': 1}
0.44045436045436037
Accuracy: 0.5484
Precision: 0.2857
Recall: 0.5000
F1 Score: 0.3636
ROC AUC: 0.5326
PR AUC: 0.2719
Fold: 3

Fitting 5 folds for each of 25 candidates, totalling 125 fits
{'learning_rate': 0.4, 'max_depth': 1}
0.4305482589069546
Accuracy: 0.6452
Precision: 0.2000
Recall: 0.1250
F1 Score: 0.1538
ROC AUC: 0.4755
PR AUC

In [ ]:
# New loop with oversampling using ImbPipeline
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

temp_result_list = []
# loop over folds
for fold, (train_idx, test_idx) in enumerate(kf.split(df['encoded_sequence'], df['quantotypic'])):
    # Splitting the data into train and test sets
    train_df = df.iloc[train_idx]
    test_df = df.iloc[test_idx]
    print(f"Fold: {fold}\n")

    # Prepare the data for training and testing
    X_train = np.stack(train_df['encoded_sequence'].values)
    X_train = X_train.reshape(X_train.shape[0], -1)
    y_train = train_df['quantotypic'].values

    X_test = np.stack(test_df['encoded_sequence'].values)
    X_test = X_test.reshape(X_test.shape[0], -1)
    y_test = test_df['quantotypic'].values
    
    # Param grid for XGBClassifier
    param_grid = {
        'classifier__max_depth': [None, 1, 3, 5, 6],  
        'classifier__learning_rate': [0.01, 0.1, 0.2, 0.3, 0.4],  
    }

    # Create a pipeline with RandomOverSampler and XGBClassifier
    pipeline = ImbPipeline([
        ('oversampler', RandomOverSampler(random_state=42)), 
        ('classifier', XGBClassifier(
            n_estimators=1000, 
            random_state=42
        ))
    ])

    # Training the model using GridSearchCV
    grid_search_xgb = GridSearchCV(
        estimator=pipeline,
        param_grid=param_grid,
        cv=5,  # Validation croisée
        scoring='average_precision',
        verbose=4,
        n_jobs=-1
    )

    grid_search_xgb.fit(X_train, y_train)

    print(grid_search_xgb.best_params_)
    print(grid_search_xgb.best_score_)

    # Retrieve the best parameters
    best_params = {k.split('__')[1]: v for k, v in grid_search_xgb.best_params_.items()}

    # Retrain the model on the entire training set with oversampling
    oversampler = RandomOverSampler(random_state=42)
    X_resampled, y_resampled = oversampler.fit_resample(X_train, y_train)

    best_xgb = XGBClassifier(
        n_estimators=1000,
        random_state=42,
        **best_params  
    )

    best_xgb.fit(X_resampled, y_resampled)

    # Predict on the test set
    y_pred = best_xgb.predict(X_test)

    # Evaluate the model using the provided eval function
    accuracy, precision, recall, f1, roc_auc, pr_auc = utils.eval(y_test, y_pred)

    metrics = {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "roc_auc": roc_auc,
        "pr_auc": pr_auc
    }

    temp_result_list.append(metrics)

    # Print the results
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"ROC AUC: {roc_auc:.4f}")
    print(f"PR AUC: {pr_auc:.4f}")

accuracy_list = [result['accuracy'] for result in temp_result_list]
precision_list = [result['precision'] for result in temp_result_list]
recall_list = [result['recall'] for result in temp_result_list]
f1_list = [result['f1'] for result in temp_result_list]
roc_auc_list = [result['roc_auc'] for result in temp_result_list]
pr_auc_list = [result['pr_auc'] for result in temp_result_list]

metrics_summary = {
    "accuracy_mean": np.mean(accuracy_list),
    "accuracy_std": np.std(accuracy_list),
    "precision_mean": np.mean(precision_list),
    "precision_std": np.std(precision_list),
    "recall_mean": np.mean(recall_list),
    "recall_std": np.std(recall_list),
    "f1_mean": np.mean(f1_list),
    "f1_std": np.std(f1_list),
    "roc_auc_mean": np.mean(roc_auc_list),
    "roc_auc_std": np.std(roc_auc_list),
    "pr_auc_mean": np.mean(pr_auc_list),
    "pr_auc_std": np.std(pr_auc_list)
}

dict_save = {
    "weighting": False,
    "oversampling": True,
    "metrics_summary": metrics_summary
}

path = "C:/Users/Walraff/OneDrive - Universite de Liege/Documents/Ulg/Master2/TFE/Results/ValidProtocol"
utils.write_into_json(dict_save, f"{path}/xgb_scratch_experiment_3.json")

Fold: 0

Fitting 5 folds for each of 25 candidates, totalling 125 fits
{'classifier__learning_rate': 0.4, 'classifier__max_depth': 1}
0.41843558029272315
Accuracy: 0.6129
Precision: 0.2727
Recall: 0.4286
F1 Score: 0.3333
ROC AUC: 0.5476
PR AUC: 0.2459
Fold: 1

Fitting 5 folds for each of 25 candidates, totalling 125 fits
{'classifier__learning_rate': 0.1, 'classifier__max_depth': 1}
0.3952860812748463
Accuracy: 0.7419
Precision: 0.4000
Recall: 0.2857
F1 Score: 0.3333
ROC AUC: 0.5804
PR AUC: 0.2756
Fold: 2

Fitting 5 folds for each of 25 candidates, totalling 125 fits
{'classifier__learning_rate': 0.01, 'classifier__max_depth': 1}
0.41908160608927875
Accuracy: 0.6129
Precision: 0.3333
Recall: 0.5000
F1 Score: 0.4000
ROC AUC: 0.5761
PR AUC: 0.2957
Fold: 3

Fitting 5 folds for each of 25 candidates, totalling 125 fits
{'classifier__learning_rate': 0.2, 'classifier__max_depth': 1}
0.42490841847363586
Accuracy: 0.6774
Precision: 0.3333
Recall: 0.2500
F1 Score: 0.2857
ROC AUC: 0.5380
PR AUC: